In [1]:
import pandas as pd
import numpy as np

Загрузим выборки.

In [2]:
train = pd.read_csv('orange_small_churn_train_data.csv', index_col = 0)
X_test = pd.read_csv('orange_small_churn_test_data.csv', index_col = 0)

In [3]:
numeric_columns = np.array(train.columns[:190])
categorical_columns = np.array(train.columns[190:230])

Для построения модели выберем только те переменные, в которых есть хотя бы одно значение, не равное NaN.

In [4]:
not_null_numeric_columns = np.array(numeric_columns[np.array(pd.notnull(train[numeric_columns]).any())])

In [5]:
not_null_categorical_columns = np.array(categorical_columns[np.array(pd.notnull(train[categorical_columns]).any())])

Для упрощения модели выберем только те категориальные признаки, в которых не больше ста различных значений.

In [6]:
categorical_less_or_equal_100 = []
for var in not_null_categorical_columns:
    if len(train[var].unique()) <= 100:
        categorical_less_or_equal_100.append(var)
categorical_less_or_equal_100 = np.array(categorical_less_or_equal_100)

In [7]:
train_numeric = train[not_null_numeric_columns]
train_categorical = train[categorical_less_or_equal_100]
y_train = train['labels']
test_numeric = X_test[not_null_numeric_columns]
test_categorical = X_test[categorical_less_or_equal_100]

Заполним пропущенные значения.

In [8]:
train_numeric = train_numeric.fillna(0)
train_categorical = train_categorical.fillna('NA')
test_numeric = test_numeric.fillna(0)
test_categorical = test_categorical.fillna('NA')

Применим к категориальным данным one-hot encoding с помощью DictVectorizer.

In [9]:
from sklearn.feature_extraction import DictVectorizer

In [10]:
dv = DictVectorizer(sparse = False)
train_categorical = dv.fit_transform(train_categorical.T.to_dict().values())
test_categorical = dv.transform(test_categorical.T.to_dict().values())

К каждому признаку добавим дополнительный признак с индикатором того, пропущено значение или нет.

In [11]:
train_numeric_is_na = pd.isnull(train[not_null_numeric_columns]).astype(int)
train_categorical_is_na = pd.isnull(train[not_null_categorical_columns]).astype(int)
test_numeric_is_na = pd.isnull(X_test[not_null_numeric_columns]).astype(int)
test_categorical_is_na = pd.isnull(X_test[not_null_categorical_columns]).astype(int)

In [12]:
X_train = np.hstack((train_numeric, train_categorical, train_numeric_is_na, train_categorical_is_na))
X_test = np.hstack((test_numeric, test_categorical, test_numeric_is_na, test_categorical_is_na))

In [13]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold

Разобьём обучающую выборку на 3 фолда методом stratified k-fold.

In [14]:
skf = StratifiedKFold(y_train, 3, random_state = 0)

В качестве модели используем градиентный бустинг (xgboost.XGBClassifier). Подбираем параметр модели learning_rate с помощью кросс-валидации и измеряем AUC-ROC.

In [15]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

In [25]:
param_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.2]}
optimizer = GridSearchCV(xgb.XGBClassifier(), param_grid, scoring = 'roc_auc', cv = skf)
optimizer.fit(X_train, y_train)
optimizer.grid_scores_

[mean: 0.70851, std: 0.00701, params: {'learning_rate': 0.01},
 mean: 0.73729, std: 0.00431, params: {'learning_rate': 0.05},
 mean: 0.74175, std: 0.00272, params: {'learning_rate': 0.1},
 mean: 0.73626, std: 0.00247, params: {'learning_rate': 0.2}]

In [26]:
optimizer.best_params_

{'learning_rate': 0.1}

Видим, что оптимальным параметром является learning_rate = 0.1. Средний AUC-ROC для него равен 0.74175.

Посчитаем вероятности на тестовой выборке.

In [27]:
probabilities = np.array([x[1] for x in optimizer.predict_proba(X_test)])

Запишем результаты в файл.

In [30]:
results = pd.DataFrame({'ID' : range(len(probabilities)), 'result' : probabilities})
results.to_csv('results.csv', index = False)